In [ ]:
#!/usr/bin/env python
# coding: utf-8

import time
import RPi.GPIO as GPIO
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
# import os

# new_model = tf.keras.models.load_model('myCNNClassificationAwakeSleep.h5')
deviceid = 0  # it depends on the order of USB connection.

# TFLiteモデルの読み込み
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")
# メモリ確保。これはモデル読み込み直後に必須
interpreter.allocate_tensors()

# 学習モデルの入力層・出力層のプロパティをGet.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def lotate(degree):
    dc = 2.5 + (12.0-2.5)/180*(degree+90)
    p.ChangeDutyCycle(dc)
    time.sleep(0.5)
    p.ChangeDutyCycle(0.0)

while True:
    try:
        capture = cv2.VideoCapture(deviceid)

        ret, frame = capture.read()
        ret, frame = capture.read()
        cv2.imwrite('./image/test11.jpg', frame)
    except:
        print("camera error")
    # os.environ['KMP_DUPLICATE_LIB_OK']='True'

    trainingImgData = np.zeros((1, 128, 128, 1))
    fnametmp = './image/test11.jpg'
    trainingImgData[0] = np.array(img_to_array(load_img(fnametmp, grayscale=True, target_size=(128,128))))/255.0
    trainingImgData = np.float32(trainingImgData)

    # #モデルに画像を代入します．
    # valigender = new_model.predict(testImgData)
    # Test the model on random input data.
    interpreter.set_tensor(input_details[0]['index'], trainingImgData)
    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    flag=0
    
    
    if output_data[0][0] > 0.5:
        flag=1
        print("sleep")
        

        GPIO.setmode(GPIO.BCM)
        #GPIO4を出力端子設定 
        GPIO.setup(4, GPIO.OUT)
        
        #GPIO4をPWM設定、周波数は50Hz
        p = GPIO.PWM(4, 50)
        
        #Duty Cycle 0% 
        p.start(0.0)
        
        #-90°の位置へ移動 
        for i in range(1):
            lotate(15)
            lotate(0)
            lotate(-15)
            lotate(0)

        GPIO.cleanup()

        

    else:
        # if flag == 1:
        #     GPIO.setmode(GPIO.BCM)
        #     #GPIO4を出力端子設定 
        #     GPIO.setup(4, GPIO.OUT)
            
        #     #GPIO4をPWM設定、周波数は50Hz 
        #     p = GPIO.PWM(4, 50)
            
        #     #Duty Cycle 0% 
        #     p.start(0.0)
            
        #     #-90°の位置へ移動 
        #     for i in range(1):
        #         lotate(15)
        #         lotate(0)
        #         lotate(-15)
        #         lotate(0)

        #     GPIO.cleanup()

        print("awake")
    time.sleep(10)

